In [4]:
import pandas as pd
import os
from datetime import datetime

In [39]:
def getFileList(srch_dir):
    '''this function finds all files in a given directory
    '''
    
    file_list = []
    for root, directories, filenames in os.walk(srch_dir):
        for filename in filenames: 
            if filename.lower()[-4:]=='.csv':
                file_list.append(os.path.join(root,filename) )
    return file_list


def generateUnixTime(row):
    '''
        function to convert string timestamp and daylist savings flag to unix time
        
        attempts to process the two datetime string formats found in the current dataset
        but will fail if any other datetime string format is present
    '''
    
    try:
        dt = datetime.strptime(row['Time'], "%m/%d/%Y %H:%M")
    except:
        try:
           dt = datetime.strptime(row['Time'], "%Y-%m-%d %H:%M")
        except:
            print('datetime conversion failed')
            print(row['Time'])
            dt = 0
    
    dt = (dt - datetime(1970,1,1)).total_seconds()
    
    # add time diff depending on DST flag
    if row['Daylight Savings Flag'] == 'Y':
        dt += 7*60*60
    elif row['Daylight Savings Flag'] == 'N':
        dt += 8*60*60
    else:
        print('missing DST flag')
    
    return dt


def extractLoadForYear(inFolder):
    '''
        function finds all csv files in given folder, reads them as panda dfs and combines them
        converts timestamp to unix time, then drops unnecessary columns
        
        preseves unix time and original timestamp and DST flag
        
        aggregates all submeters into single total load amount
        
    '''
    

    # look in directory for load files
    fileList = getFileList(inFolder)

    # intialize dataframe var
    results = None

    # loop through found files
    for ff in fileList:
        
        print('reading: %s' %ff)
        
        # read files in list to dataframe
        df = pd.read_csv(ff)
        
        
        # drop and process columns

        # add to cumulative dataframe
        if results is None:
            print('initializing df with %s' %ff)
            results = df
        else:
            print('ading data from %s' %ff)
            results = pd.concat([results, df])
            
    return results


def processLoadData(inFolder):

    # extract and combine load data from files in inFolder
    results = extractLoadForYear(inFolder)

    # generate unix timestamp
    # this operation seems pretty inefficient, but not sure of a better way
    # since unix time requires multiple fields to calaculate
    results['unixTime'] = results.apply(generateUnixTime, axis=1)
    
    # copy to results raw, to extract datetime
    resultsRaw = results.copy(deep=True)
    resultsRaw = resultsRaw[['unixTime', 'Time', 'Daylight Savings Flag']]
    resultsRaw.columns = ['timestamp', 'datetime', 'dst']
    
    # drop rows and sum by timestamp
    results = results[['unixTime', 'Usage Value']]
    results = results.groupby(['unixTime'], as_index=False)['Usage Value'].sum()
    
    # convert timestep energy to power
    results.columns = ['timestamp', 'load']
    results.load = results.load * 4
    
    # add timestamp back to results df
    rRight = results.set_index('timestamp')

    rLeft = resultsRaw.drop_duplicates(subset=['timestamp'])
    rLeft = rLeft.set_index('timestamp')

    r2 = rRight.join(rLeft, how='left')
 
    return r2

def processLoadDataMeter(inFolder):

    # extract and combine load data from files in inFolder
    results = extractLoadForYear(inFolder)

    # generate unix timestamp
    # this operation seems pretty inefficient, but not sure of a better way
    # since unix time requires multiple fields to calaculate
    results['unixTime'] = results.apply(generateUnixTime, axis=1)
    
    # select and rename relevant columns
    results = results[['unixTime', 'Time', 'Daylight Savings Flag', 'Meter Badge Number', 'Usage Value']]
    results.columns = ['timestamp', 'datetime', 'dst', 'meter', 'load']
    # convert timestep energy to power
    results.load = results.load * 4
    
    # copy to results raw, to extract and reapply datetime/dst flag
    resultsRaw = results.copy(deep=True)
    resultsRaw = resultsRaw[['timestamp', 'datetime', 'dst']]
    
    # pivot results to make meter loads columns
    results = results[results.load!=0]
    results.reset_index(inplace=True)
    results = results[['timestamp', 'meter', 'load']]
    results = results.pivot(index='timestamp', columns='meter')

    results.columns = results.columns.droplevel(0) #remove load multiIndex
    results.columns.name = None               #remove categories
    results = results.reset_index()                #index to columns


    
    # add timestamp back to results df
    rRight = results.set_index('timestamp')

    rLeft = resultsRaw.drop_duplicates(subset=['timestamp'])
    rLeft = rLeft.set_index('timestamp')

    r2 = rRight.join(rLeft, how='left')
 
    return r2


def combineWeather(leftData, rightData):
    '''
        function combines load data and weather data by matching closest unix timestamp
        returns data for time index of leftData
    '''
    
    # sort both by index
    leftData = leftData.sort_index()
    rightData = rightData.sort_index()
    
    # convert both indices to int
    leftData.index = leftData.index.astype(int)
    rightData.index = rightData.index.astype(int)
    
    combinedData = pd.merge_asof(leftData, rightData, on="timestamp")
    
    return combinedData



In [5]:
# extract weather data from files
w19 = pd.read_csv('/Users/nxd/Downloads/Parks Data/weather data/Camp Parks 2019 weather data.csv')
w20 = pd.read_csv('/Users/nxd/Downloads/Parks Data/weather data/Camp Parks 2020 weather data.csv')

# set timestamp as index
w19 = w19.set_index('timestamp')
w20 = w20.set_index('timestamp')

# define directories for 2019 and 2020 data
dir2019 = '/Users/nxd/Downloads/Parks Data/load data/2019'
dir2020 = '/Users/nxd/Downloads/Parks Data/load data/2020'

# process load data for 2019
# r19 = processLoadData(dir2019)

# proces load data for 2020
# r20 = processLoadData(dir2020)

In [10]:
# # create weather-load tables for 2019 and 2020 for 15min and 1hr timesteps
# # then write to file
# c19 = combineWeather(r19, w19)
# print(c19.shape)
# c19.to_csv('/Users/nxd/Downloads/Parks Data/training data/Parks_2019_training_data_15min.csv')

# c19b = combineWeather(w19, r19)
# print(c19b.shape)
# c19b.to_csv('/Users/nxd/Downloads/Parks Data/training data/Parks_2019_training_data_1hr.csv')

# c20 = combineWeather(r20, w20)
# print(c20.shape)
# c20.to_csv('/Users/nxd/Downloads/Parks Data/training data/Parks_2020_training_data_15min.csv')

# c20b = combineWeather(w20, r20)
# print(c20b.shape)
# c20b.to_csv('/Users/nxd/Downloads/Parks Data/training data/Parks_2020_training_data_1hr.csv')


In [41]:
# code to generate meter-specific load training data
m19 = processLoadDataMeter(dir2019)
m20 = m19 = processLoadDataMeter(dir2020)

# create weather-load tables for 2019 and 2020 for 15min and 1hr timesteps
# then write to file
c19 = combineWeather(m19, w19)
print(c19.shape)
c19.to_csv('/Users/nxd/Downloads/Parks Data/training data/Parks_2019_training_data_METER_15min.csv')

c19b = combineWeather(w19, m19)
print(c19b.shape)
c19b.to_csv('/Users/nxd/Downloads/Parks Data/training data/Parks_2019_training_data_METER_1hr.csv')

c20 = combineWeather(m20, w20)
print(c20.shape)
c20.to_csv('/Users/nxd/Downloads/Parks Data/training data/Parks_2020_training_data_METER_15min.csv')

c20b = combineWeather(w20, m20)
print(c20b.shape)
c20b.to_csv('/Users/nxd/Downloads/Parks Data/training data/Parks_2020_training_data_METER_1hr.csv')

reading: /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20190901-20190930.csv
initializing df with /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20190901-20190930.csv
reading: /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20191101-20191130.csv
ading data from /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20191101-20191130.csv
reading: /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20191201-20191231.csv
ading data from /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20191201-20191231.csv
reading: /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20190201-20190228.csv
ading data from /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20190201-20190228.csv
reading: /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20191001-20191031.csv
ading data from /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20191001-20191031.csv
reading: /

In [16]:
# extract raw data to explore meter-level completeness
rawData2019 = extractLoadForYear(dir2019)
rawData2020 = extractLoadForYear(dir2020)

reading: /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20190901-20190930.csv
initializing df with /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20190901-20190930.csv
reading: /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20191101-20191130.csv
ading data from /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20191101-20191130.csv
reading: /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20191201-20191231.csv
ading data from /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20191201-20191231.csv
reading: /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20190201-20190228.csv
ading data from /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20190201-20190228.csv
reading: /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20191001-20191031.csv
ading data from /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20191001-20191031.csv
reading: /

In [19]:
rawData2019.groupby('Meter Badge Number')['Usage Value'].agg(['count', 'sum'])

,count,sum
Meter Badge Number,,
1004578059,70080,5.408226e+06
1006706934,35040,1.752400e+05
1006715799,35040,4.886148e+04
1006716429,35040,1.870230e+05
1006727093,35040,4.746609e+05
1006727361,35040,5.866292e+04
1008913626,35040,4.938008e+03
1009074083,35040,2.518137e+03
1009511425,35040,4.046645e+05


In [18]:
rawData2020.groupby('Meter Badge Number')['Usage Value'].agg(['count', 'sum'])

,count,sum
Meter Badge Number,,
1004578059,69888,4.910166e+06
1006706934,35136,2.156694e+05
1006715799,35136,5.066628e+04
1006716429,35136,1.917336e+05
1006727093,35136,4.824462e+05
1006727361,35136,4.504716e+04
1008913626,35136,6.848189e+03
1009074083,35136,1.527174e+03
1009511425,35136,4.005050e+05


In [40]:
x = processLoadDataMeter(dir2019)
# x2 = x.copy(deep=True)
x.head()

reading: /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20190901-20190930.csv
initializing df with /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20190901-20190930.csv
reading: /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20191101-20191130.csv
ading data from /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20191101-20191130.csv
reading: /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20191201-20191231.csv
ading data from /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20191201-20191231.csv
reading: /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20190201-20190228.csv
ading data from /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20190201-20190228.csv
reading: /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20191001-20191031.csv
ading data from /Users/nxd/Downloads/Parks Data/load data/2019/Parks_Historical_20191001-20191031.csv
reading: /

,1004578059,1006706934,1006715799,1006716429,1006727093,1006727361,1008913626,1009074083,1009511425,1009540871,1009984043,1009984060,1010054507,1010086328,1010249061,1010282897,1010397607,1010409824,datetime,dst
timestamp,,,,,,,,,,,,,,,,,,,,
1.546330e+09,558.0,18.56,5.44,33.60,33.28,0.96,0.796,0.2376,26.88,8.16,3.244,22.348,8.96,76.48,22.8,NaN,NaN,24.96,1/1/2019 0:15,N
1.546331e+09,552.6,18.56,4.80,33.28,32.64,0.64,0.792,0.8652,22.40,8.16,3.988,21.956,9.92,76.48,31.2,NaN,NaN,33.92,1/1/2019 0:30,N
1.546332e+09,549.6,18.24,4.80,33.28,31.36,1.28,0.800,0.9020,24.32,9.12,3.124,22.076,10.24,72.32,27.6,NaN,NaN,28.80,1/1/2019 0:45,N
1.546333e+09,562.2,18.56,4.80,33.28,32.64,1.28,0.796,0.3804,23.68,8.64,3.292,23.784,10.24,71.68,25.2,NaN,NaN,24.96,1/1/2019 1:00,N
1.546334e+09,549.6,18.24,5.12,33.28,35.20,0.96,0.792,0.7524,23.68,8.64,3.964,24.944,9.28,72.64,32.4,NaN,NaN,32.64,1/1/2019 1:15,N


In [20]:
x2.shape

(646854, 5)

In [36]:
x2 = x.copy(deep=True)
x2 = x2[x2.load!=0]
x2.reset_index(inplace=True)
x2 = x2[['timestamp', 'meter', 'load']]
x2 = x2.pivot(index='timestamp', columns='meter')

x2.columns = x2.columns.droplevel(0) #remove load multiIndex
x2.columns.name = None               #remove categories
x2 = x2.reset_index()                #index to columns

# print(x.shape)
print(x2.shape)

x2.head()


(35040, 19)


,timestamp,1004578059,1006706934,1006715799,1006716429,1006727093,1006727361,1008913626,1009074083,1009511425,1009540871,1009984043,1009984060,1010054507,1010086328,1010249061,1010282897,1010397607,1010409824
0,1.546330e+09,558.0,18.56,5.44,33.60,33.28,0.96,0.796,0.2376,26.88,8.16,3.244,22.348,8.96,76.48,22.8,NaN,NaN,24.96
1,1.546331e+09,552.6,18.56,4.80,33.28,32.64,0.64,0.792,0.8652,22.40,8.16,3.988,21.956,9.92,76.48,31.2,NaN,NaN,33.92
2,1.546332e+09,549.6,18.24,4.80,33.28,31.36,1.28,0.800,0.9020,24.32,9.12,3.124,22.076,10.24,72.32,27.6,NaN,NaN,28.80
3,1.546333e+09,562.2,18.56,4.80,33.28,32.64,1.28,0.796,0.3804,23.68,8.64,3.292,23.784,10.24,71.68,25.2,NaN,NaN,24.96
4,1.546334e+09,549.6,18.24,5.12,33.28,35.20,0.96,0.792,0.7524,23.68,8.64,3.964,24.944,9.28,72.64,32.4,NaN,NaN,32.64


In [35]:
x2.columns = x2.columns.droplevel(0) #remove amount
x2.columns.name = None               #remove categories
x2 = x2.reset_index()                #index to columns
x2.head()

,timestamp,1004578059,1006706934,1006715799,1006716429,1006727093,1006727361,1008913626,1009074083,1009511425,1009540871,1009984043,1009984060,1010054507,1010086328,1010249061,1010282897,1010397607,1010409824
0,1.546330e+09,558.0,18.56,5.44,33.60,33.28,0.96,0.796,0.2376,26.88,8.16,3.244,22.348,8.96,76.48,22.8,NaN,NaN,24.96
1,1.546331e+09,552.6,18.56,4.80,33.28,32.64,0.64,0.792,0.8652,22.40,8.16,3.988,21.956,9.92,76.48,31.2,NaN,NaN,33.92
2,1.546332e+09,549.6,18.24,4.80,33.28,31.36,1.28,0.800,0.9020,24.32,9.12,3.124,22.076,10.24,72.32,27.6,NaN,NaN,28.80
3,1.546333e+09,562.2,18.56,4.80,33.28,32.64,1.28,0.796,0.3804,23.68,8.64,3.292,23.784,10.24,71.68,25.2,NaN,NaN,24.96
4,1.546334e+09,549.6,18.24,5.12,33.28,35.20,0.96,0.792,0.7524,23.68,8.64,3.964,24.944,9.28,72.64,32.4,NaN,NaN,32.64
